An ***interface*** is similar to a class, but only `specifies behavior` and `does not hold state (data)`. Consequently:  
- An interface can define `only functions` and `not fields`.  
  
- Interface members are `implicitly abstract`. (Although nonabstract functions are permitted from C# 8, this is considered a special case)  
  
- A class (or struct) `can implement multiple interfaces`. In contrast, a class can inherit from `only a single class`, and a struct `cannot inherit at all` (aside from deriving from System.ValueType).

An interface can contain ***only functions***, that is, `methods`, `properties`, `events`, and `indexers`

Interface members are `always implicitly public` and `cannot declare an access modifier`. Implementing an interface means `providing a public implementation` `for all of its members`:

In [ ]:
public interface IEnumerator
{
    bool MoveNext();
    object Current { get; }
    void Reset();
}

internal class Countdown : IEnumerator
{
    int count = 11;
    public bool MoveNext() => count-- > 0;
    public object Current => count;
    public void Reset() { throw new NotSupportedException(); }
}

//You can implicitly cast an object to any interface that it implements:

IEnumerator e = new Countdown();

### Extending an Interface

In [ ]:
//Interfaces can derive from other interfaces; for instance:

public interface IUndoable { void Undo(); }
public interface IRedoable : IUndoable { void Redo(); }

### Explicit Interface Implementation

In [ ]:
//Implementing multiple interfaces 
//can sometimes result in a collision between member signatures.

interface I1 { void Foo(); }
interface I2 { int Foo(); }
public class Widget : I1, I2
{
    public void Foo()
    {
        Console.WriteLine ("Widget's implementation of I1.Foo");
    }
    int I2.Foo() //explicitly implementing
    {
        Console.WriteLine ("Widget's implementation of I2.Foo");
        return 42;
    }
}

Widget w = new Widget();
w.Foo(); // Widget's implementation of I1.Foo
((I1)w).Foo(); // Widget's implementation of I1.Foo
((I2)w).Foo(); // Widget's implementation of I2.Foo

### Implementing Interface Members Virtually

An implicitly `implemented` interface member is, `by default`, `sealed`.

In [ ]:
//It must be marked virtual or abstract in the base class in order to be overridden
public interface IUndoable { void Undo(); }
public class TextBox : IUndoable
{
    public virtual void Undo() => Console.WriteLine ("TextBox.Undo");
    //must be virtual for overriding
}
public class RichTextBox : TextBox
{
    public override void Undo() => Console.WriteLine ("RichTextBox.Undo");
}

//Calling the interface member through either 
//the base class or the interface calls the subclass’s implementation:
RichTextBox r = new RichTextBox();
r.Undo();                   // RichTextBox.Undo
((IUndoable)r).Undo();      // RichTextBox.Undo
((TextBox)r).Undo();        // RichTextBox.Undo


### Reimplementing an Interface in a Subclass

In [ ]:
//Reimplementation hijacks a member implementation (when called through
//the interface) and works whether or not the member is virtual in the base class.

public interface IUndoable { void Undo(); }
public class TextBox : IUndoable
{
    void IUndoable.Undo() => Console.WriteLine ("TextBox.Undo");
}
public class RichTextBox : TextBox, IUndoable
{
    public void Undo() => Console.WriteLine ("RichTextBox.Undo");
}

RichTextBox r = new RichTextBox();
r.Undo();               // RichTextBox.Undo Case 1
((IUndoable)r).Undo();  // RichTextBox.Undo Case 2

In [ ]:
public interface IUndoable { void Undo(); }

public class TextBox : IUndoable
{
    public void Undo() => Console.WriteLine ("TextBox.Undo"); //implicitly
}

public class RichTextBox : TextBox, IUndoable
{
    public void Undo() => Console.WriteLine ("RichTextBox.Undo");
}



### Default Interface Members

In [ ]:
//From C# 8, you can add a default implementation to an interface member, 
//making it optional to implement:

interface ILogger
{
    void Log (string text) => Console.WriteLine (text);
}

class Logger : ILogger { }

var log = new Logger();

log.Log() //error // just call through the interface

ILogger log1 = new Logger();
log1.Log("");


In [ ]:
//Interfaces can also now define static members (including fields), which can be
//accessed from code inside default implementations

interface ILogger
{
    void Log (string text) =>
        Console.WriteLine (Prefix + text);

    static string Prefix = "";
}

//Because interface members are implicitly public, 
//you can also access static members from the outside
ILogger.Prefix = "File log: ";